In [2]:
import pandas as pd
import numpy as np

In [3]:
df_normal = pd.read_csv("archive/ptbdb_normal.csv", header = None)
df_abnormal = pd.read_csv("archive/ptbdb_abnormal.csv", header = None)
df = pd.concat([df_normal, df_abnormal])
print(df.shape)

(14552, 188)


In [4]:
# Separating features and target 
X = np.array(df[df.columns[:-1]])
print(f'The feature space is of shape: {X.shape}')
Y= np.array(df[187])
print(f'The target is of shape: {Y.shape}')

The feature space is of shape: (14552, 187)
The target is of shape: (14552,)


In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import recall_score as recall
from sklearn.metrics import precision_score as precision
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [11]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)
# Make predictions with those same features
y_kmeans = kmeans.predict(X)

In [12]:
y_kmeans

array([0, 1, 0, ..., 0, 0, 0], dtype=int32)

In [14]:
cm= confusion_matrix(Y,y_kmeans )
acc  = accuracy(Y,y_kmeans)
rec  = recall(Y,y_kmeans )
prec = precision(Y,y_kmeans )
f1   = f1_score(Y,y_kmeans )
print(f'Accuracy = {acc}, Precision = {prec}, Recall = {rec}, F1-score = {f1}')
print('Confusion Matrix is:')
print(cm)

Accuracy = 0.4777350192413414, Precision = 0.8391690009337068, Recall = 0.34218541785646295, F1-score = 0.48613928329952666
Confusion Matrix is:
[[3357  689]
 [6911 3595]]


In [15]:
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
cm= confusion_matrix(Y,labels )
acc  = accuracy(Y,labels)
rec  = recall(Y,labels)
prec = precision(Y,labels)
f1   = f1_score(Y,labels )
print(f'Accuracy = {acc}, Precision = {prec}, Recall = {rec}, F1-score = {f1}')
print('Confusion Matrix is:')
print(cm)

/Users/kinga/anaconda3/envs/kaggleECG/lib/python3.6/site-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Accuracy = 0.2795492028587136, Precision = 1.0, Recall = 0.0020940415000951836, F1-score = 0.004179331306990881
Confusion Matrix is:
[[ 4046     0]
 [10484    22]]


In [16]:
labels 


array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [19]:
switched = -1*(labels -1)
switched

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [20]:
cm= confusion_matrix(Y,switched )
acc  = accuracy(Y,switched)
rec  = recall(Y,switched)
prec = precision(Y,switched)
f1   = f1_score(Y,switched )
print(f'Accuracy = {acc}, Precision = {prec}, Recall = {rec}, F1-score = {f1}')
print('Confusion Matrix is:')
print(cm)

Accuracy = 0.7204507971412865, Precision = 0.7215416379903647, Recall = 0.9979059584999048, F1-score = 0.8375139798689887
Confusion Matrix is:
[[    0  4046]
 [   22 10484]]


In [21]:
from pyod.models.knn import KNN
from pyod.utils.utility import standardizer

In [ ]:
def get_knn_scores_and_predictions(X_train, X_test, n_neighbors=5):
    ### START CODE HERE ###
    # Initialize the model
    clf = KNN(n_neighbors=n_neighbors)
    # Train the model
    clf.fit(X_train)
    # Get the training data's outlier/decision scores with the decision_scores_ attribute
    y_train_scores = clf.decision_scores_
    # Generate the anomaly scores for the test data using the decision_function() method 
    y_test_scores = clf.decision_function(X_test)
    # Convert the outlier scores to 2D column arrays, then standardize them so they have 
    # a mean of 0 and standard deviation of 1
    y_train_scores_norm, y_test_scores_norm = standardizer(y_train_scores.reshape(-1,1)), standardizer(y_test_scores.reshape(-1,1))
    # Apply the model to the test data to get predictions
    y_test_pred = clf.predict(X_test) # outlier labels (0 or 1)
    ### END CODE HERE ###
    return y_train_scores_norm, y_test_scores_norm, y_test_pred

# Call the function with the default KNN
y_train_scores_norm, y_test_scores_norm, y_test_pred = get_knn_scores_and_predictions(X_train, X_test)